In [1]:
from pandas import DataFrame
from DATA225utils import make_connection, dataframe_query

In [2]:
def make_table(table, sql):
    cursor_warehouse.execute(f"DROP TABLE IF EXISTS {table}")
    cursor_warehouse.execute(sql)

In [3]:
def display_table(table, order_by=''):
    sql = f"SELECT * FROM {table}"
    
    if order_by != '':
        sql = sql + " ORDER BY " + order_by
        
    _, df = dataframe_query(conn_warehouse, sql)    
    return df

## Sources: the operational tables

In [24]:
conn_sales = make_connection(config_file = 'ecommerce.ini')
cursor_sales = conn_sales.cursor()


## The dimensional model (data warehouse)

In [25]:
conn_warehouse = make_connection(config_file = 'ecommerce_wh.ini')
cursor_warehouse = conn_warehouse.cursor()

## Drop tables

In [26]:
cursor_warehouse.execute('DROP TABLE IF EXISTS Review')

In [27]:
cursor_warehouse.execute('DROP TABLE IF EXISTS Sales')

In [28]:
cursor_warehouse.execute('DROP TABLE IF EXISTS Customer_D')

In [29]:
cursor_warehouse.execute('DROP TABLE IF EXISTS Customer_Address_D')

In [30]:
cursor_warehouse.execute('DROP TABLE IF EXISTS Order_D')

In [31]:
cursor_warehouse.execute('DROP TABLE IF EXISTS Product_D')

## Product dimension

In [32]:
sql = ( """
        CREATE TABLE Product_D
        (
          ProductId INT NOT NULL,
  ProductName VARCHAR(512) NOT NULL,
  ProductPrice NUMERIC(4,2) NOT NULL,
  InStock CHAR(1) NOT NULL,
  CategoryId INT NOT NULL,
  CategoryName VARCHAR(64) NOT NULL,
  SubCategoryId INT NOT NULL,
  SubCategoryName VARCHAR(64) NOT NULL,
  PRIMARY KEY (ProductId)
        )
        """
      )

make_table('Product_D', sql)

In [33]:
sql = ( """
        INSERT INTO ecommerce_wh.Product_D(ProductID, ProductName,ProductPrice,InStock,CategoryId,CategoryName,SubCategoryId,SubCategoryName)
        SELECT DISTINCT p.productId, p.productName, p.productPrice, p.InStock, c.id, c.name, s.id, s.name
        FROM  ecommerce.Products p, ecommerce.Category c, ecommerce.Sub_Category s
        WHERE p.CategoryId = c.Id
        AND p.SubCategoryId = s.Id

        """
      )

cursor_sales.execute(sql)
conn_sales.commit()

display_table('Product_D')

,ProductId,ProductName,ProductPrice,InStock,CategoryId,CategoryName,SubCategoryId,SubCategoryName
0,1,Chocolate Sandwich Cookies,3.49,1,19,snacks,61,cookies cakes
1,2,All-Seasons Salt,5.99,1,13,pantry,104,spices seasonings
2,3,Robust Golden Unsweetened Oolong Tea,8.29,1,7,beverages,94,tea
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,5.99,1,1,frozen,38,frozen meals
4,5,Green Chile Anytime Sauce,6.29,1,13,pantry,5,marinades meat preparation
...,...,...,...,...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",14.79,1,5,alcohol,124,spirits
49684,49685,En Croute Roast Hazelnut Cranberry,13.79,1,1,frozen,42,frozen vegan vegetarian
49685,49686,Artisan Baguette,4.29,1,3,bakery,112,bread
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,25.99,1,8,pets,41,cat food care


## Customer dimension

In [34]:
sql = ( """
        CREATE TABLE Customer_D
        (
          CustomerId INT NOT NULL,
  FirstName VARCHAR(32) NOT NULL,
  LastName VARCHAR(32) NOT NULL,
  Email VARCHAR(128) NOT NULL,
  Age INT NOT NULL,
  Gender CHAR(1) NOT NULL,
  Phone VARCHAR(10) NOT NULL,
  PRIMARY KEY (CustomerId)
        )
        """
      )

make_table('customer', sql)

In [35]:
sql = ( """
        INSERT INTO ecommerce_wh.Customer_D(CustomerID, FirstName,LastName,Email,
        Age, Gender,Phone)
            SELECT DISTINCT CustomerId, FirstName,LastName,Email,Age,
            Gender,Phone
            FROM ecommerce.Customer
            
        """
      )

cursor_sales.execute(sql)
conn_sales.commit()
display_table('Customer_D')

,CustomerId,FirstName,LastName,Email,Age,Gender,Phone
0,1001,Joella,Schimpke,jschimpke0@opensource.org,31,F,2179406269
1,1002,Faith,Balston,fbalston1@spotify.com,24,F,6127842194
2,1003,Ginnifer,Hestrop,ghestrop2@reuters.com,67,F,2565828554
3,1004,Crin,Cowan,ccowan3@canalblog.com,42,F,4141344859
4,1005,Orran,Gambrell,ogambrell4@oaic.gov.au,68,M,8506961701
...,...,...,...,...,...,...,...
12700,13701,Adolphus,Marfield,amarfieldjg@prnewswire.com,39,M,7028804720
12701,13702,Wolf,Gonzalvo,wgonzalvojh@wix.com,77,M,5014953362
12702,13703,Marketa,Pleasants,mpleasantsji@webmd.com,20,F,7024504435
12703,13704,Shepperd,Milkin,smilkinjj@google.com,66,M,4848332959


## Customer_Address Dimension

In [36]:
sql = ( """
         CREATE TABLE Customer_Address_D
        (
          AddressId INT NOT NULL,
  AptNo VARCHAR(32),
  City VARCHAR(32) NOT NULL,
  State VARCHAR(32) NOT NULL,
  PinCode INT NOT NULL,
  StreetName VARCHAR(32) NOT NULL,
  PRIMARY KEY (AddressId)
        )
        """
      )

make_table('Customer_Address_D', sql)

In [37]:
sql = ( """
        INSERT INTO ecommerce_wh.Customer_Address_D(AddressId,AptNo,City,State,
        PinCode, StreetName)
            SELECT DISTINCT AddressId,AptNo,City,State,PinCode,StreetName
            FROM ecommerce.Customer_Address
            
        """
      )

cursor_sales.execute(sql)
conn_sales.commit()
display_table('Customer_Address_D')

,AddressId,AptNo,City,State,PinCode,StreetName
0,1,,Springfield,Illinois,62723,7472 3rd Place
1,2,Room 148,Minneapolis,Minnesota,55448,04844 Bunker Hill Road
2,3,Suite 32,Gadsden,Alabama,35905,91 Jackson Court
3,4,Suite 21,Milwaukee,Wisconsin,53210,778 Ryan Junction
4,5,,Pensacola,Florida,32526,65 Grim Court
...,...,...,...,...,...,...
12700,12701,PO Box 2500,Las Vegas,Nevada,89120,68896 Anniversary Road
12701,12702,Room 547,Little Rock,Arkansas,72222,3 Evergreen Crossing
12702,12703,Room 1004,Las Vegas,Nevada,89150,153 Killdeer Trail
12703,12704,Suite 93,Valley Forge,Pennsylvania,19495,7446 Marcy Circle


## Orders Dimension

In [38]:
sql = ( """
         CREATE TABLE Order_D
(
  OrderId INT NOT NULL,
  DayofWeek INT NOT NULL,
  OrderDate DATE NOT NULL,
  PRIMARY KEY (OrderId)
)

        """
      )

make_table('Orders', sql)

In [39]:
sql = ( """
        INSERT INTO ecommerce_wh.Order_D(OrderId, DayofWeek,OrderDate)
            SELECT DISTINCT OrderId, DayofWeek,OrderDate
            FROM ecommerce.Orders
            
        """
      )

cursor_sales.execute(sql)
conn_sales.commit()
display_table('Order_D')

,OrderId,DayofWeek,OrderDate
0,10000,7,2022-02-19
1,10001,1,2020-12-13
2,10002,4,2023-02-22
3,10003,7,2020-09-19
4,10004,1,2020-04-26
...,...,...,...
17313,27313,6,2020-07-31
17314,27314,1,2020-07-26
17315,27315,6,2022-07-29
17316,27316,7,2020-07-04


## Sales fact table

In [40]:
sql = ( """
        CREATE TABLE sales
        (
          Quantity INT NOT NULL,
  Amount FLOAT NOT NULL,
  AddressId INT NOT NULL,
  OrderId INT NOT NULL,
  CustomerId INT NOT NULL,
  ProductId INT NOT NULL,
  FOREIGN KEY (AddressId) REFERENCES Customer_Address_D(AddressId),
  FOREIGN KEY (OrderId) REFERENCES Order_D(OrderId),
  FOREIGN KEY (CustomerId) REFERENCES Customer_D(CustomerId),
  FOREIGN KEY (ProductId) REFERENCES Product_D(ProductId)
)
        """
      )

make_table('sales', sql)

In [41]:
sql = ( """
        INSERT INTO ecommerce_wh.sales(Quantity,Amount,AddressId,OrderId,CustomerId,ProductId)
        
       SELECT od.Quantity, p.ProductPrice * od.Quantity, ca.AddressId, o.OrderId, o.CustomerId, od.ProductId
        FROM ecommerce.Order_Details od
        JOIN ecommerce.Products p ON od.ProductId = p.ProductId
        JOIN ecommerce.Orders o ON od.OrderId = o.OrderId
        JOIN ecommerce.Customer_Address ca ON o.CustomerId =ca.CustomerId;

        """
      )

cursor_sales.execute(sql)
conn_sales.commit()
display_table('sales')

,Quantity,Amount,AddressId,OrderId,CustomerId,ProductId
0,2,13.98,1,10061,1001,25056
1,3,17.97,1,18962,1001,7061
2,1,5.29,1,18962,1001,14239
3,4,25.16,1,18962,1001,29503
4,2,12.98,1,19621,1001,42092
...,...,...,...,...,...,...
69015,1,8.29,12704,15445,13704,5589
69016,2,8.58,12704,15445,13704,12458
69017,1,7.49,12704,15445,13704,42913
69018,2,4.58,12705,15906,13705,34481


## Reviews fact table

In [42]:
sql = ( """
        CREATE TABLE Review
(
  Rating INT NOT NULL,
  Description VARCHAR(512),
  ProductId INT NOT NULL,
  CustomerId INT NOT NULL,
  FOREIGN KEY (ProductId) REFERENCES Product_D(ProductId),
  FOREIGN KEY (CustomerId) REFERENCES Customer_D(CustomerId)
)
    """
      )

make_table('Review', sql)

In [43]:
sql = ( """
        INSERT INTO ecommerce_wh.Review(Rating,Description,ProductID, CustomerId)
            SELECT Ratings,Description,ProductID,CustomerId
            FROM ecommerce.Reviews
            
        """
      )

cursor_sales.execute(sql)
conn_sales.commit()

display_table('Review')

,Rating,Description,ProductId,CustomerId
0,1,The packaging on this Chocolate Sandwich Cooki...,1,5211
1,4,I've been buying Chocolate Sandwich Cookies fr...,1,12644
2,4,I'm really happy with my Chocolate Sandwich Co...,1,9710
3,5,I'm pleasantly surprised with how good these C...,1,9064
4,4,I'm really happy with my All-Seasons Salt purc...,2,3588
...,...,...,...,...
124166,5,Smartblend Healthy Metabolism Dry Cat Food is ...,49687,3674
124167,3,I was really looking forward to this Smartblen...,49687,3821
124168,4,I'm pleasantly surprised with how good these S...,49687,13571
124169,3,I was really disappointed with this Smartblend...,49687,11832


In [44]:
cursor_sales.close()
conn_sales.close()

cursor_warehouse.close()
conn_warehouse.close()